**Imports**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import missingno as msno
import import_ipynb
import UTILS as utils
  
import FundamentalAnalysis as fa

importing Jupyter notebook from UTILS.ipynb


In [3]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [4]:
api_key = "0165bc7584022886e8fd42cfc99473a2"

---------------

In [6]:
file1 = 'DATA_ttm/S_P500_data_ttm_0yeardisp/S_P500_dataframe_with_ratios_ttm_0yeardisp.csv'
file2 = 'DATA_ttm/S_P500_data_ttm_0yeardisp/S_P500_companies_financials_data_ttm_0yeardisp.pickle'

In [7]:
data_loaded = utils.data_loading(file1, file2)

In [8]:
df = data_loaded[0]
dict_companies = data_loaded[1]
df.drop(["_asset_to_equity","_total_debt_to_capitalization"], axis=1, inplace=True)

In [ ]:
#Effectively, when loading the dataframe we don't get None anymore and these are replaced by NaNs
df.applymap(lambda x: x is None).eq(False).all()

In [14]:
df.head(10)

,_PAT_margin,_EBITDA_margin,_return_on_equity,_return_on_assets,_return_on_capital_employed,_gross_margin,_operating_margin,_operating_cash_flow_ratio,_return_on_invested_capital,_return_on_sales,_fixed_asset_turnover,_working_capital_turnover,_total_asset_turnover,_inventory_turnover,_days_of_inventory_on_hand,_accounts_receivables_turnover,_payables_turnover,_debt_to_assets,_interest_coverage,_debt_to_equity,_equity_multiplier,_total_debt_to_capital,_net_debt_to_EBITDA,_degree_financial_leverage,_earnings_per_share,_book_value_per_share,_cash_earnings_per_share,_price_to_earnings,_price_to_book_value,_price_to_sales,_price_to_free_cash_flow,_price_earnings_to_growth,_dividend_yield,_dividend_payout,_enterprise_value_to_ebitda,_enterprise_value_to_sales,_current_ratio,_quick_ratio,_cash_ratio,_days_of_sales_outstanding,_days_of_sales_in_inventory,_days_of_payables_outstanding,_operating_cycle,_cash_conversion_cycle
ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MMM,0.167402,0.271661,0.102917,0.031030,0.047709,0.468315,0.208277,2.300579,0.047709,0.208277,0.866823,5.229256,0.185262,0.978114,92.077553,1.813772,1.642398,0.696076,15.093071,1.176688,3.309888,0.566805,5.917859,2.226105,2.555000,24.922730,3.219128,18.235344,7.483685,12.189879,73.837823,-2.092910,0.007957,0.580389,50.806934,3.449114,1.743940,1.115540,0.515781,49.624411,201.254556,54.859324,141.701964,86.842640
AOS,0.137160,0.199539,0.065875,0.037287,0.066318,0.370851,0.171587,21.825725,0.066318,0.171587,1.483506,5.073510,0.271250,1.547255,58.337771,1.472644,0.873009,0.431103,143.894444,0.082905,1.761376,0.079406,-1.930628,1.033130,0.757969,11.493758,1.000958,23.926370,6.235913,13.052287,98.040415,1.010740,0.003732,0.355069,63.643361,3.159238,1.736831,1.302403,0.518312,61.207901,248.232043,103.114310,119.545672,16.431362
ABT,0.163348,0.284307,0.051291,0.023943,0.034499,0.568713,0.194551,1.643727,0.034499,0.194551,1.215246,4.189462,0.145944,0.872851,103.387451,1.396316,1.124048,0.530724,15.826453,0.514217,2.145139,0.350867,3.222767,0.996922,0.998719,19.428942,1.487627,32.244209,6.357053,20.308296,102.341309,-2.211618,0.003678,0.476043,75.751684,5.305609,1.804959,1.343743,0.700832,65.792770,266.826236,80.302048,169.180221,88.878173
ABBV,0.205323,0.426591,0.204293,0.019435,0.038300,0.688963,0.318653,0.674716,0.038300,0.318653,2.729707,-15.077336,0.094679,1.355834,66.467690,1.450563,1.498959,0.906772,7.421584,5.237012,10.811499,0.854522,12.574431,234.068885,1.633018,7.814001,3.222009,27.050788,14.846429,14.581623,39.208925,-0.058520,0.011391,1.242405,48.412273,4.916135,0.886588,0.642223,0.326778,62.202190,252.264436,15.010417,128.669880,113.659463
ABMD,0.133839,0.217640,0.023884,0.021451,0.024020,0.817721,0.138994,173.757015,0.024020,0.138994,1.230112,1.437473,0.165194,0.536747,167.934020,2.759614,1.478162,0.102285,NaN,0.000000,1.113981,0.000000,15.859374,0.021046,0.732500,30.051012,1.613612,21.090068,10.937711,59.488986,251.044773,-2.858338,NaN,NaN,-1273.585663,14.679971,6.796166,5.789275,1.653611,32.744503,132.797150,61.164649,200.678522,139.513873
ACN,0.112673,0.184220,0.080035,0.036871,0.076610,0.323523,0.151342,1.366361,0.076610,0.151342,2.950586,13.179046,0.326960,NaN,0.000000,1.355853,4.450394,0.526363,156.535289,0.137940,2.172496,0.149194,-1.494978,0.753381,2.522500,31.474604,2.966377,32.151081,10.247288,14.438970,231.055434,-0.754573,0.002885,0.368570,77.092586,3.539411,1.281334,1.160854,0.478060,66.433258,269.423771,20.324177,66.433258,46.109081
ATVI,0.305766,0.417571,0.040978,0.028187,0.037962,0.736469,0.370010,8.509178,0.037962,0.370010,12.487110,0.986028,0.091795,NaN,0.000000,3.018355,2.389189,0.311327,31.476631,0.218610,1.453404,0.179324,-6.809646,1.320549,0.867500,21.280114,0.776645,24.200490,3.937931,29.256733,121.961211,-0.214270,NaN,NaN,63.557279,6.646529,4.758636,4.346914,4.037153,30.798070,124.903284,37.904457,30.798070,-7.106387
ADM,0.031813,0.053505,0.031196,0.012836,0.022247,0.070600,0.035271,1.099264,0.022247,0.035271,1.961318,7.994280,0.404290,

In [ ]:
df.info()

----------

In [ ]:
###Get descriptive statistics

df.describe()

------------------------

In [ ]:
###Plot distributions for each ratio


fig, axes = plt.subplots(nrows=len(df.columns[1:]), ncols=2, figsize=(18, len(df.columns[1:])*6))
plt.subplots_adjust(hspace=0.5)

for yax in range(len(df.columns[1:])):
    for xax in range(2):
        sns.kdeplot(df[df.columns[yax+1]].values, shade=False, color='lightskyblue', ax=axes[yax, xax])
        kdeline = axes[yax, xax].lines[0]
        xs = kdeline.get_xdata()
        ys = kdeline.get_ydata()
        if xax == 0:
            middle = df[df.columns[yax+1]].values.mean()
            sdev = df[df.columns[yax+1]].values.std()
            left = middle - sdev
            right = middle + sdev
            axes[yax, xax].set_title(f'{df.columns[yax+1]} - showing mean and sdev')
        else:
            left, middle, right = np.percentile(df[df.columns[yax+1]].values, [25, 50, 75])
            axes[yax, xax].set_title(f'{df.columns[yax+1]} - showing median and quartiles')
        axes[yax, xax].vlines(middle, 0, np.interp(middle, xs, ys), color='lightskyblue', ls=':')
        axes[yax, xax].fill_between(xs, 0, ys, facecolor='lightskyblue', alpha=0.2)
        axes[yax, xax].fill_between(xs, 0, ys, where=(left <= xs) & (xs <= right), interpolate=True, facecolor='lightskyblue', alpha=0.2)
        # ax.set_ylim(ymin=0)
plt.show()

-------------------

In [ ]:
###Identify correlations among ratios


plt.figure(figsize= [25,18])
corr = df[df.columns[1:]].corr().round(2)
sns.heatmap(corr, annot=True, annot_kws={"fontsize":8}, cmap = sns.diverging_palette(220, 20, as_cmap=True), vmin=-1, vmax=1)

-----------------

**Sectorial identification and analysis**

In [16]:
sectors_df, industries_df = utils.get_company_sectors_and_industries(df, dict_companies)

In [17]:
if set(['sector','industry']).issubset(df.columns) == False:
    df.insert(0, "sector", sectors_df.sector)
    df.insert(1, "industry", industries_df.industry)

In [18]:
sectors_df.sector.unique()        #11 sectors

array(['Industrials', 'Healthcare', 'Technology',
       'Communication Services', 'Consumer Defensive',
       'Consumer Cyclical', 'Utilities', 'Financial Services',
       'Basic Materials', 'Real Estate', 'Energy'], dtype=object)

In [ ]:
df.sector.value_counts().plot(kind='barh', figsize=(8,4))

In [78]:
industries_df.industry.unique()    #115 industries

array(['Conglomerates', 'Specialty Industrial Machinery',
       'Medical Devices', 'Drug Manufacturers—General',
       'Information Technology Services',
       'Electronic Gaming & Multimedia', 'Farm Products',
       'Software—Infrastructure', 'Staffing & Employment Services',
       'Specialty Retail', 'Utilities—Diversified', 'Insurance—Life',
       'Diagnostics & Research', 'Insurance—Diversified',
       'Specialty Chemicals', 'Airlines', 'REIT—Office',
       'Security & Protection Services', 'Utilities—Regulated Electric',
       'Insurance—Property & Casualty', 'Internet Content & Information',
       'Tobacco', 'Internet Retail', 'Packaging & Containers',
       'Semiconductors', 'Credit Services', 'REIT—Specialty',
       'Utilities—Regulated Water', 'Asset Management',
       'Medical Distribution', 'Electronic Components',
       'Software—Application', 'Healthcare Plans', 'Insurance Brokers',
       'Oil & Gas E&P', 'Consumer Electronics',
       'Semiconductor Equipme

In [ ]:
df.industry.value_counts()

In [ ]:
df.groupby('sector').industry.nunique().sort_values(ascending=False).plot(kind='barh', figsize=(8,4))
plt.locator_params(axis="x", nbins=39)

In [ ]:
df.industry.value_counts().plot(kind='bar', figsize=(20,4))
plt.locator_params(axis="y", nbins=39)

In [ ]:
 pd.pivot_table(data=df.reset_index(), index=['sector', 'industry'], values=['ticker'], aggfunc='count').rename(columns={'ticker': 'number of tickers'})

**Company type analysis**

29 of the tickers classified as trust are REITs (real estate sector) and 1 belongs to asset management industry (financial services sector)

There's one ticker (CBRE) belonging to the 'Real Estate' sector that is considered as an stock

--------------------

**Analyse missing data**

In [ ]:
print(len(df.columns[df.isna().any()])) #Number of colums with at least one missing value

print(len(df.columns[df.notna().all()]))#Number of colums without any missing value

In [ ]:
###Number of null values for each ratio


num_nulls_by_ratio = df.isnull().sum(axis=0)

num_nulls_by_ratio.plot.bar(figsize=(20,8))

In [ ]:
for i in range(df.shape[1]):
    n_miss = df[df.columns[i]].isnull().sum(axis=0)
    perc = n_miss / df.shape[0] * 100
    print(f'{df.columns[i]} ---- Missing: {n_miss} ({round(perc,2)}%)')

In [ ]:
###Number of null values for each ticker (i.e. company)


num_nulls_by_ticker = df.isnull().sum(axis=1).sort_values(ascending=False)

num_nulls_by_ticker[:40].plot.bar(figsize=(20,8))  #it used to be figsize=(8,100)
plt.locator_params(axis="y", nbins=20)

In [ ]:
len(df) - len(df[df.isna().any(axis=1)])

In [ ]:
len(df[df.isna().any(axis=1)])

In [ ]:
### What would happen if we dropped all tickers that contain NaN values at some column?

df.dropna().info()

In [ ]:
combinations=[]
for i in df.columns:
    ratios_to_compare = df.columns.tolist()
    ratios_to_compare.remove(i)
    for r in ratios_to_compare:
        if (df[df[i].isnull()].index.tolist() == df[df[r].isnull()].index.tolist()) and len(df[df[i].isnull()].index.tolist())!=0:
            combinations.append([i, r])
unique_combinations = {frozenset(x) for x in combinations}
[list(x) for x in unique_combinations]

In [ ]:
msno.matrix(df, labels=True)

In [ ]:
msno.heatmap(df)

In [ ]:
### Do we have infinities? No, good

np.inf in df.values or -np.inf in df.values